# DSGE and State-Space Modeling with Julia
*2017 Reserve Bank Workshops*

** *Erica Moszkowski* ** *(@emoszkowski)*, <br/>
***Federal Reserve Bank of New York*** *(@FRBNY-DSGE)*

*3/10/2017 - 3/13/2017*

## Disclaimer

This talk reflects the experience of the speaker with Julia and MATLAB and does
not represent an endorsement by the Federal Reserve Bank of New York or the
Federal Reserve System of any particular product or service. The views
expressed in this talk are those of the speaker and do not necessarily reflect
the position of the Federal Reserve Bank of New York or the Federal Reserve System.

## Outline

- Quick review of DSGEs
- DSGE.jl: from MATLAB to Julia
  - How?
  - Why? 
- Code design approaches in DSGE.jl
- More useful tools you can use: StateSpaceRoutines.jl
- Exercise

## Collaborators

- FRBNY-DSGE team: 
  - Marco Del Negro, Marc Giannoni, Abhi Gupta, Pearl Li, Sara Shahanaghi, Micah Smith
- QuantEcon collaborators:
  - Zac Cranko, Spencer Lyon, John Stachurski, Pablo Winant

## The DSGE model 

A DSGE model is a system of dynamic equations: 

<center>$ \Gamma_0(\theta) s_{t} = \Gamma_1(\theta) s_{t-1} + \Gamma_C(\theta) + \Psi(\theta) \epsilon_{t} + \Pi(\theta) \eta_{t} $ &emsp; $\epsilon_t \sim N(0,Q(\theta))$</center> 

where
- $s_t$ is a vector of states at time $t$ (capital, inflation, ...)
  - includes forecasts of future states
- $\epsilon_t$ is a vector of shocks (productivity, financial,...)
- $\eta_t$ is a vector of forecast errors (under rational expectations in the current version of the FRBNY DSGE model)
- $\theta$ is a vector of parameters describing preferences and technology (depreciation rate, ...)

## The DSGE model (ctd) 

The model solution (given by `gensys`) is the *transition equation*:

<center>$ s_{t}  = T(\theta) s_{t-1} + R(\theta) \epsilon_{t} + C(\theta) $ &emsp; &emsp; &emsp; (1)</center> 

The *measurement equation* maps states to observed data ($y_t$): 

<center>$ y_{t}  = Z(\theta) s_{t} + D(\theta)$ &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; (2) </center>

Equations (1) and (2) form a state-space system.

**A DSGE model is a *mapping* between $\theta$ and $\Gamma_0, \Gamma_1, \Gamma_C, \Psi, \Pi$ (which map to $T, R, C$), $Q, Z$, and $D$.**

- This mapping is needed to compute the likelihood $p(y|\theta$) from the state-space model. 
  - The likelihood is the probability of observing the data given a set of parameters.
  - *Preview*: `DSGE.jl` centers around a "model object" that stores this mapping. 

- We adopt a Bayesian approach: $\theta$ is a random variable, and we impose a prior $p(\theta)$ on it.
   - The `DSGE.jl` model object also includes information on this prior.

- Once we know $\theta$, we know everything -> $\theta$ is the key variable of interest

## Estimating the parameter values

- Goal: conduct inference about $\theta$ and $s_t$  (obtain draws from the posterior distribution $p(\theta|y) \propto p(y|\theta)p(\theta))$

- How? Sample from posterior distribution using `metropolis_hastings` MCMC method.
  - This is the most computationally expensive part of estimation (takes ~10 hrs)

In [ ]:
for i = 1:120000   # metropolis_hastings *very* computationally intensive

    draw θ_i
    solve(model, θ_i)                       # `gensys`
    loglike = likelihood(model, data, θ_i)  # `kalman_filter` => p(data | θ_i)
    logpost = loglike + logprior(θ_i)       # compute p(θ_i | data)
    
end

## Forecasting states and observables 

Iterate state-space system forward from $T$ (end of sample) to forecast future values of states and observables:

$$ s_{T+1}  = T(\theta) s_{T} + R(\theta) \epsilon_{T+1} + C(\theta) $$
$$ y_{T+1}  = Z(\theta) s_{T+1} + D(\theta) $$ <br>

We can do this for $1,2,\dots,h$ periods ahead, iterating forward one period at a time.

## `DSGE.jl`

- Julia package that facilitates the solution and Bayesian estimation of DSGE models
<br>
- Users specify their own models and apply our implementation of workhorse procedures
<br>
- Main sections:
    1. Prepare data 
      - Integrates with FredData.jl to pull from Federal Reserve Economic Data (FRED)
    2. Solve model and estimate time-invariant parameters 
    3. Forecast macro variables (e.g. GDP growth, inflation) 
    4. Plot (not yet released)
    5. ...

## `using DSGE`

In [ ]:
using DSGE

# construct a model object
m = Model990()

# estimate as of 2015-Q3 using the default data vintage from 2015 Nov 27
m <= Setting(:data_vintage, "151127")
m <= Setting(:date_mainsample_end, quartertodate("2015-Q3"))

# loads data from FRED and user-supplied CSV files
df = load_data(m)   

# reoptimize parameter vector, compute Hessian at mode, and full posterior
# parameter sampling
estimate(m, df)

# produce LaTeX tables of parameter moments
compute_moments(m)

## From MATLAB to Julia: The Story

**~10 years ago** &emsp;  &emsp;  DSGE team begins! 

**September 2014** &emsp;   MATLAB codebase made public (in .zip) on *Liberty Street Economics* 

**Spring 2015** &emsp; &emsp; &emsp;  QuantEcon approaches DSGE team  

<ul style="margin-top:0px;"><li>For QuantEcon</li>
  <ul><li>Add production DSGE model from central bank to collection of high-quality open-source code for economics</li></ul></ul>
  
<ul style="margin-top:0px;"><li>For DSGE team</li>
  <ul><li>Improve performance sufficiently to experiment with new techniques (e.g. nonlinear models)</li>
  <li>Redesign codebase from bottom up</li>
  <li>Make code free!</li></ul></ul>
  
**December 2015** &emsp;   Release of DSGE.jl (estimation step)

**June 22, 2016** &emsp; &emsp; Release of data step (v0.1.3) 

**March/April, 2017** &emsp; Release of forecast step (v0.2.0) <br>
        &emsp; &emsp; &emsp; Release of StateSpaceRoutines.jl <br>
        &emsp; &emsp; &emsp; Release of BVARs.jl 

## Why Julia?  

**Initial reasoning**
  - A bit of a push from John and QuantEcon (thank you!)
  - Some Julia features make it easier for MATLAB users to pick up (1-indexing, for example)

**With some reflection**

A number of Julia <font color="blue">features</font> lend themselves to DSGE.jl <font color="green">implementation</font>

  - Central <font color="blue">type system</font> provides a natural way to <font color="green">organize</font> and <font color="green">simplify</font> codebase
  - <font color="blue">Method dispatch</font> allows us to write more <font color="green">general</font> code
  - Rely on the <font color="blue">compiler</font> to boost <font color="green">performance</font>

**Challenges to be aware of**

- New language: frequent updates. This will slow down when v1.0 comes out (hopefully this year)

- Sparse StackOverflow activity

*Keep in mind:* DSGE.jl is not a direct port! Even so...some motivating stats:

## Types + dispatch = simpler code

<br>

| Language             | Lines of code (hundreds) |
| -------------------- | :----------------------: |
| Matlab               | 63                       |
| Julia                | 37                       |

<center> Lines of code used in estimation step. *(Including redesign)* 

## <font color="green">Performance</font> improvements

No substantial redesign:

| Test                  | MATLAB (14a) | Julia (0.4.0)  |
| --------------------  | :----------: | :------------: |
| `gensys`              | 1.00         | 5.88           |
| `solve`               | 1.00         | 11.11          |
| `kalman_filter`       | 1.00         | 1.33           |
| `posterior`           | 1.00         | 3.85           |
| `csminwel`            | 1.00         | 3.03           |
| `hessian`             | 1.00         | 4.35           |


Substantial redesign:

| Test                  | MATLAB (14a) | Julia (0.4.0)  |
| --------------------  | :----------: | :------------: |
| `metropolis_hastings` | 1.00         | 9.09           |

<center>Benchmark speedup relative to MATLAB (larger is better) </center>

<center>*On an Intel® Xeon® E5-2697 v2 2.70GHz CPU running GNU/Linux*</center>

## DSGE.jl design: types all the way down

- In Julia, it's *natural* to take a type-oriented approach to an economic model 


- Types allow for intuitive expression of economic objects (e.g. models, parameters)
   - Encapsulate all features of that object in a single data structure
   - Seamless dispatch to support economic intuition
   
   
  
- Applications in `DSGE.jl`: 
   - `AbstractModel`
   - `Parameters`
   - `Settings`

In [3]:
using DSGE, Distributions

INFO: Recompiling stale cache file /Users/ericamoszkowski/.julia/lib/v0.4/HDF5.ji for module HDF5.
INFO: Recompiling stale cache file /Users/ericamoszkowski/.julia/lib/v0.4/Blosc.ji for module Blosc.
INFO: Recompiling stale cache file /Users/ericamoszkowski/.julia/lib/v0.4/DSGE.ji for module DSGE.
INFO: Recompiling stale cache file /Users/ericamoszkowski/.julia/lib/v0.4/Roots.ji for module Roots.
INFO: Recompiling stale cache file /Users/ericamoszkowski/.julia/lib/v0.4/Polynomials.ji for module Polynomials.
INFO: Recompiling stale cache file /Users/ericamoszkowski/.julia/lib/v0.4/ForwardDiff.ji for module ForwardDiff.
INFO: Recompiling stale cache file /Users/ericamoszkowski/.julia/lib/v0.4/QuantEcon.ji for module QuantEcon.
INFO: Recompiling stale cache file /Users/ericamoszkowski/.julia/lib/v0.4/DSP.ji for module DSP.
INFO: Recompiling stale cache file /Users/ericamoszkowski/.julia/lib/v0.4/LightGraphs.ji for module LightGraphs.
INFO: Recompiling stale cache file /Users/ericamoszkows

## Case study #1: `Parameter` types

Recall that DSGE parameters (θ) aren't just values. They have a number of features:

In [4]:
type Parameter{T<:Number}
    key::Symbol
    value::T                        # parameter value 
    valuebounds::Tuple{T,T}         # bounds of parameter value
    prior::Distribution             # prior distribution
    fixed::Bool                     # is this parameter fixed at some value?
    description::AbstractString
    tex_label::AbstractString       # LaTeX label for printing
end

α = Parameter(:α, 0.1596, (1e-5, 0.999), Normal(0.30, 0.05),  
              false, "α: Capital share", "\\alpha")

Parameter{Float64}(:α,0.1596,(1.0e-5,0.999),Distributions.Normal{Float64}(μ=0.3, σ=0.05),false,"α: Capital share","\\alpha")

## Case study #1 (ctd):  multiple dispatch supports economic intuition

We should be able to have α + 1: 

In [5]:
Base.(:+)(p::Parameter, x::Number) = p.value + x
α + 1.

1.1596

We should also be able to add parameters on the right: 1 + α

In [6]:
Base.(:+)(x::Number, p::Parameter) = p+x
1. + α   # has meaning! 

1.1596

The ease with which we can do this is fairly unique to Julia, or at least languages with multiple dispatch.

## Case study #1 (ctd)  

Why is parameter arithmetic interesting? 

Because we can define equilibrium conditions using parameter algebra! 

In [ ]:
y_t = Φ*(α*k_t + (1-α)*L_t)   # Log-linearized production function 

This is a only *slight* simplification of how a user would implement their equilibrium conditions and measurement equation.

## Case study #2a: Dealing with computational settings

- "Computational settings" = flags or numbers without economic intiution
- There are *many* of these in a production DSGE model like ours:
  - Can we read in precomputed modal parameters, or should we optimize?
  - How many draws should take from the posterior distribution in Metropolis-Hastings?
  - Where do we store input/output files?
- Don't want to have to track them down somewhere in the source code to change their values
- Want to deal with all settings uniformly

In [7]:
type Setting{T}
    key::Symbol                  # name of setting
    value::T                     # whatever the setting is
    print::Bool                  # whether or not to add this setting to the print
    code::AbstractString         # what gets printed to the print
    description::AbstractString  # description of what the setting is for
end

- Store a `Dict{Setting}` within the model object 

## Case study #2b:  using `Settings` to prevent file collisions

- **Problem:**
  - We create a lot of output files (particularly in `forecast`)
  - Don't want to overwrite files when using different `Setting`s

- **Solution:**
  - `print` and `code` fields of `Setting` object are used to construct a file string 

In [6]:
to_filestring(s::Setting) = "$(s.code)=$(s.value)"

to_filestring (generic function with 1 method)

## Case study #2b (ctd):  using `Settings` to prevent file collisions

In [8]:
# Setup (within model constructor)
settings = Dict{Symbol,Setting}()             

Dict{Symbol,Setting{T}} with 0 entries

In [9]:
# Construct settings
settings[:vint] = Setting(:data_vintage, "151127", 
                          true, "vint", "input data vintage")
settings[:cond] = Setting(:cond_vintage, "151205", 
                          true, "cond", "conditional data vintage")

Setting{ASCIIString}(:cond_vintage,"151205",true,"cond","conditional data vintage")

In [10]:
# When constructing filenames
filestrings = Vector{ASCIIString}()
for (skey, sval) in settings
    if sval.print
        push!(filestrings, "$(sval.code)=$(sval.value)") 
    end
end

join(filestrings,'_')                           # join all filestrings together

"cond=151205_vint=151127"

## Case study #2b (ctd): using `Settings` to prevent file collisions

- Approach to `Setting`s and filestrings is not unique to Julia
- There are probably ways to do it better
- But these are challenges economists face **all the time**

## `StateSpaceRoutines.jl`

The *transition equation* maps state yesterday ($s_{t-1}$) to state today ($s_t$):

<center>$ s_{t}  = T(\theta) s_{t-1} + R(\theta) \epsilon_{t} + C(\theta) $ &emsp; &emsp; &emsp; (1)</center> 

The *measurement equation* maps states to observed data ($y_t$): 

<center>$ y_{t}  = Z(\theta) s_{t} + D(\theta)$ &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; (2) </center>

- State space routines are not only used to estimate and forecast DSGE models 
- Don't want others to waste time implementing these routines again
- `StateSpaceRoutines.jl` provides model-agnostic state-space routines (inputs and outputs are just matrices)
  - `DSGE.jl` uses `StateSpaceRoutines.jl` as a dependency.

## `StateSpaceRoutines.jl`: Available routines

- Kalman filter (`kalman_filter`): computes log-likelihood and filtered states $s_{t|1:t}$ for $t \in 1:T$

- Kalman smoothers: computes smoothed states $s_{t|1:T}$ for $t \in 1:T$
  + `hamilton_smoother`: James Hamilton, [_Time Series Analysis_](https://www.amazon.com/Time-Analysis-James-Douglas-Hamilton/dp/0691042896) (1994)
  + `koopman_smoother`: S.J. Koopman, ["Disturbance Smoother for State Space Models"](https://www.jstor.org/stable/2336762) (_Biometrika_, 1993)

- Simulation smoothers:
  + `carter_kohn_smoother`: C.K. Carter and R. Kohn, ["On Gibbs Sampling for State Space Models"](https://www.jstor.org/stable/2337125) (_Biometrika_, 1994)
  + `durbin_koopman_smoother`: J. Durbin and S.J. Koopman, ["A Simple and Efficient Simulation Smoother for State Space Time Series Analysis"](https://www.jstor.org/stable/4140605) (_Biometrika_, 2002)


## Ongoing and future work

### Ongoing
- Plotting
- VAR package

### Future directions
- Nonlinear solution methods
- New optimization methods
  - E.g. `csminwel` => simulated annealing?
- New estimation methods
  - E.g. `metropolis_hastings` => Sequential Monte Carlo

## Example state-space model: Univariate autoregressive process

The model:

<center>$ y_{t+1} = ϕ_{1} y_t + ϕ_{2} y_{t−1} + ϕ_{3} y_{t−2} + ϕ_{4} y_{t−3} + σ w_{t+1}$ &emsp; &emsp;   $w_t \sim N(0,1)$  &emsp; &emsp; &emsp; &emsp;</center>



State-space representation:

Let $s_t$ = [$y_t$  $y_{t−1}$  $y_{t−2}$  $y_{t−3}$]′   

[insert example here]

**We are sure there are better ways to do just about anything in DSGE.jl and StateSpaceRoutines.jl - we welcome your pull requests and thoughts!**

## Thank you!  

<br/ >
https://github.com/FRBNY-DSGE/DSGE.jl/<br/>
@emoszkowski  &emsp; erica.moszkowski@gmail.com <br/>
@FRBNY-DSGE  &emsp; ny.dsge.ny@ny.frb.org 